# Simple Imitation Learning in MineRL
This tutorial contains a simple example of how to build a imitation-learning based agent that can solve the MineRLNavigateDense-v0 environment. For more information about that environment, see this [MineRL Docs](http://minerl.io/docs/environments/index.html#minerlnavigatedense-v0).

For more Imitation Learning algorithms, like a Dagger in Tensorflow, see that Github repo, [Dagger](https://github.com/zsdonghao/Imitation-Learning-Dagger-Torcs).

Parts of this tutorial are based on code by [Arthur Juliani](https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-2-ded33892c724).

In [1]:
from __future__ import division

import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
%matplotlib inline
import matplotlib.pyplot as plt
import math

try:
    xrange = xrange
except:
    xrange = range
    
env_name = 'MineRLTreechop-v0'
data_path = '/media/kimbring2/6224AA7924AA5039/minerl_data'

/home/kimbring2/minerl_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kimbring2/minerl_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kimbring2/minerl_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kimbring2/minerl_env/lib/python3.6/site-packages/tensorflow/pyth

### Setting up our Neural Network agent
This time we will be using a Policy neural network that takes observations, passes them through a single hidden layer, and then produces a probability of choosing a left/right movement. To learn more about this network, see [Andrej Karpathy's blog on Policy Gradient networks](http://karpathy.github.io/2016/05/31/rl/).

In [2]:
H = 1024

tf.reset_default_graph()

if (env_name == 'MineRLTreechop-v0'):
    state = tf.placeholder(shape=[None,64,64,3], dtype=tf.float32)
elif (env_name == 'MineRLNavigateDense-v0'):
    state = tf.placeholder(shape=[None,64,64,4], dtype=tf.float32)

conv1 = slim.conv2d(inputs=state, num_outputs=32, kernel_size=[8,8], stride=[4,4], padding='VALID', 
                    biases_initializer=None, activation_fn=tf.nn.relu)
conv2 = slim.conv2d(inputs=conv1, num_outputs=64, kernel_size=[4,4], stride=[2,2], padding='VALID', 
                    biases_initializer=None, activation_fn=tf.nn.relu)
conv3 = slim.conv2d(inputs=conv2, num_outputs=64, kernel_size=[3,3], stride=[1,1], padding='VALID', 
                    biases_initializer=None, activation_fn=tf.nn.relu)

convFlat = slim.flatten(conv3)
#print("convFlat: " + str(convFlat))

if (env_name == 'MineRLTreechop-v0'):
    W = tf.get_variable("W", shape=[H,6],
               initializer=tf.contrib.layers.xavier_initializer())
    score = tf.matmul(convFlat, W)
    probability = tf.nn.softmax(score)
    real_action = tf.placeholder(shape=[None,6], dtype=tf.int32)
elif (env_name == 'MineRLNavigateDense-v0'):
    W = tf.get_variable("W", shape=[H,4],
               initializer=tf.contrib.layers.xavier_initializer())
    score = tf.matmul(convFlat, W)
    probability = tf.nn.softmax(score)
    real_action = tf.placeholder(shape=[None,4], dtype=tf.int32)
    
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=real_action, 
                                                              logits=score))
tf.summary.scalar('loss', loss)
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)

merged = tf.summary.merge_all()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



# Train
MineRL package provides a human playing dataset for improving effiency of traning. At first, we are going to train our network by this dataset and use pretrained network for Reinforcement Learning. I assure it will reduce traing time tremendously. 

For more information about that dataset, see this [MineRL Dataset Docs](http://minerl.io/docs/tutorials/data_sampling.html).

### Running the Agent and Environment

In [11]:
import minerl
data = minerl.data.make(env_name, data_path)

print("test")

init = tf.global_variables_initializer()
restore = False
with tf.Session() as sess:
    rendering = False
    sess.run(init)
    saver = tf.train.Saver(max_to_keep=5)
    train_writer = tf.summary.FileWriter('/home/kimbring2/MineRL/train_summary/' + env_name, sess.graph)
    
    if restore == True:
        path = '/home/kimbring2/MineRL/model/' + env_name
        ckpt = tf.train.get_checkpoint_state(path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    
    episode_count = 0
    for current_state, action, reward, next_state, done in data.sarsd_iter(num_epochs=500, max_sequence_len=10):
        #print("action: " + str(action))
        
        #print("current_state['equipped_items']: " + str(current_state['equipped_items']))
        length = (current_state['pov'].shape)[0]

        action_list = []
        states_list = []
        for i in range(0, length):
            if (env_name == 'MineRLTreechop-v0'):
                state_concat = current_state['pov'][i].astype(np.float32) / 255.0 - 0.5
            elif (env_name == 'MineRLNavigateDense-v0'):
                pov = current_state['pov'][i].astype(np.float32) / 255.0 - 0.5
                compass = current_state['compassAngle'][i]
                #print("compass: " + str(compass))
                
                #print("np.ones(shape=list(pov.shape[:-1]): " + str(np.ones(shape=list(pov.shape[:-1]))))
                compass_channel = np.ones(shape=list(pov.shape[:-1]) + [1], dtype=np.float32) * compass
                #print("compass_channel: " + str(compass_channel))
                
                compass_channel /= 180.0
                
                state_concat = np.concatenate([pov, compass_channel], axis=-1)

                
            if (env_name == 'MineRLNavigateDense-v0'):
                if (action['camera'][i][1] < 0):
                    action_ = [1, 0, 0, 0]
                elif (action['camera'][i][1] > 0):
                    action_ = [0, 1, 0, 0]
                else:
                    if (action['jump'][i] == 0):
                        action_ = [0, 0, 1, 0]
                    else:
                        action_ = [0, 0, 0, 1]
            elif (env_name == 'MineRLTreechop-v0'):
                if (action['camera'][i][1] < 0):
                    action_ = [1, 0, 0, 0, 0, 0]
                elif (action['camera'][i][1] > 0):
                    action_ = [0, 1, 0, 0, 0, 0]
                elif (action['camera'][i][0] < 0):
                    action_ = [0, 0, 1, 0, 0, 0]
                elif (action['camera'][i][0] > 0):
                    action_ = [0, 0, 0, 1, 0, 0]
                else:
                    if ( (action['jump'][i] == 0) & (action['forward'][i] == 0) ):
                        action_ = [0, 0, 0, 0, 1, 0]
                    elif ( (action['jump'][i] == 1) & (action['forward'][i] == 1) ):
                        action_ = [0, 0, 0, 0, 0, 1]                    
                                                       
            states_list.append(state_concat)
            action_list.append(action_)
        
        episode_count = episode_count + 1
        
        #state_train = (np.zeros([1,H]), np.zeros([1,H]))
        feed_dict = {state:np.stack(states_list, 0),
                     real_action:np.stack(action_list, 0)
                    }
        
        if episode_count % 100 == 0:
            summary, _ = sess.run([merged, train_step], feed_dict=feed_dict)
            train_writer.add_summary(summary, episode_count)

        sess.run(train_step, feed_dict=feed_dict)
        
        if episode_count % 100 == 0:
            model_path = '/home/kimbring2/MineRL/model/' + env_name
            saver.save(sess, model_path + '/model-' + str(episode_count) + '.cptk')
            print("Saved Model")

test
action: OrderedDict([('attack', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('back', array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1])), ('camera', array([[ -0.1500001 ,  -0.74998474],
       [ -0.        ,  -1.0499878 ],
       [ -0.        ,  -5.25      ],
       [ -0.5999999 , -15.450012  ],
       [ -0.9       , -14.100006  ],
       [ -0.6       ,  -4.799988  ],
       [ -0.        ,   1.3500061 ],
       [ -0.        ,  13.500015  ],
       [  0.6       ,  23.399994  ],
       [  0.3       ,  29.399986  ]], dtype=float32)), ('craft', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('equip', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('forward', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('jump', array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1])), ('left', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('nearbyCraft', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('nearbySmelt', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('place', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('right', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])), ('sneak', arr

action: OrderedDict([('attack', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('back', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('camera', array([[ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.44999886, -0.        ],
       [ 1.1999979 ,  0.7500076 ],
       [ 0.8999996 ,  1.3500023 ],
       [ 0.44999886,  1.3500023 ],
       [ 0.7499981 ,  1.3500023 ],
       [ 1.199997  ,  1.5       ],
       [ 0.5999985 ,  1.5000038 ],
       [ 0.5999985 ,  0.90000534]], dtype=float32)), ('craft', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('equip', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('forward', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])), ('jump', array([0, 0, 0, 0, 0, 0, 1, 1, 1, 0])), ('left', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('nearbyCraft', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('nearbySmelt', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('place', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('right', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('sneak', array([0, 0, 0, 0, 0, 0, 0, 

action: OrderedDict([('attack', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])), ('back', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('camera', array([[  0.       ,   0.       ],
       [ -1.2000008,   0.5999994],
       [ -4.800003 ,  -1.0500011],
       [-16.199999 ,  -1.5      ],
       [-26.1      ,  -5.4000006],
       [-15.75     ,  -0.3000002],
       [-13.049999 ,  -0.       ],
       [ -8.249996 ,   0.6000004],
       [ -4.200001 ,   0.1500001],
       [ -0.6000023,  -0.       ]], dtype=float32)), ('craft', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('equip', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('forward', array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0])), ('jump', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('left', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('nearbyCraft', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('nearbySmelt', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('place', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('right', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('sneak', array([0, 0, 0, 0, 0, 0, 0, 

action: OrderedDict([('attack', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])), ('back', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('camera', array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)), ('craft', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('equip', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('forward', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('jump', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('left', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('nearbyCraft', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('nearbySmelt', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('place', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('right', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('sneak', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('sprint', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))])
action: OrderedDict([('attack', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])), ('back', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0

action: OrderedDict([('attack', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('back', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('camera', array([[ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [-0.        ,  0.5999985 ],
       [-0.        ,  2.8500013 ],
       [-0.14999962,  7.949997  ],
       [-0.        ,  7.9500027 ],
       [-0.        ,  1.0499992 ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ]], dtype=float32)), ('craft', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('equip', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('forward', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])), ('jump', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('left', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('nearbyCraft', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('nearbySmelt', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('place', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('right', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('sneak', array([0, 0, 0, 0, 0, 0, 0, 

...

Process ForkPoolWorker-82:
Process ForkPoolWorker-83:
Process ForkPoolWorker-84:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kimbring2/minerl_env/lib/python3.6/site-packages/minerl/data/data_pipeline.py", line 416, in _load_data_pyfunc
    data_queue.put(

action: OrderedDict([('attack', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])), ('back', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('camera', array([[ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [-0. , -4.5]], dtype=float32)), ('craft', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('equip', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('forward', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('jump', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('left', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('nearbyCraft', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('nearbySmelt', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('place', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('right', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('sneak', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), ('sprint', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))])
action: OrderedDict([('attack', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])), ('ba

  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)


KeyboardInterrupt: 

KeyboardInterrupt


Here we run the neural network agent, and have it act in the CartPole environment.

In [ ]:
'''
import minerl
import gym
env = gym.make('MineRLNavigateDense-v0')

obs  = env.reset()
done = False
net_reward = 0

while not done:
    action = env.action_space.noop()

    action['camera'] = [0, -10]
    action['back'] = 0
    action['forward'] = 1
    action['jump'] = 1
    action['attack'] = 1

    obs, reward, done, info = env.step(
        action)

    net_reward += reward
    print("Total reward: ", net_reward)
'''

In [ ]:
#import minerl
#data = minerl.data.make('MineRLNavigateDense-v0', '/home/kimbring2/MineRL/data/')

# Test

In [3]:
import minerl
import gym

env = gym.make('MineRLTreechop-v0')
obs = env.reset()

In [14]:
import random

init = tf.global_variables_initializer()
with tf.Session() as sess:
# Launch the graph
    rendering = False
    sess.run(init)
    saver = tf.train.Saver(max_to_keep=5)
    train_writer = tf.summary.FileWriter('/home/kimbring2/MineRL/train_summary', sess.graph)
    
    print('Loading Model...')
    path = '/home/kimbring2/MineRL/model/' + env_name
    ckpt = tf.train.get_checkpoint_state(path)
    saver.restore(sess, ckpt.model_checkpoint_path)
    
    env.init()
    obs = env.reset()
    net_reward = 0
    for i in range(0, 500000):
        if (env_name == 'MineRLTreechop-v0'):
            state_concat = obs['pov'].astype(np.float32) / 255.0 - 0.5
        elif (env_name == 'MineRLNavigateDense-v0'):
            pov = obs['pov'].astype(np.float32) / 255.0 - 0.5
            compass = obs['compassAngle']
            compass_channel = np.ones(shape=list(pov.shape[:-1]) + [1], dtype=np.float32) * compass
            compass_channel /= 180.0
            state_concat = np.concatenate([pov, compass_channel], axis=-1)
        
        e = 0.01
        
        action_probability = sess.run(probability, feed_dict={state:[state_concat]})
        if np.random.rand(1) >= e:
            action_index = np.argmax(action_probability)
        else:
            if (env_name == 'MineRLNavigateDense-v0'):
                action_index = random.randint(0,4)
            elif (env_name == 'MineRLTreechop-v0'):
                action_index = random.randint(0,6)
            
        action = env.action_space.noop()
        if (env_name == 'MineRLNavigateDense-v0'):
            if (action_index == 0):
                action['camera'] = [0, -10]
                action['jump'] = 0
                action['forward'] = 1
                action['sprint'] = 1
            elif (action_index == 1):
                action['camera'] = [0, 10]
                action['jump'] = 0
                action['forward'] = 1
                action['sprint'] = 1
            elif (action_index == 2):
                action['camera'] = [0, 0]
                action['jump'] = 0
                action['forward'] = 1
                action['sprint'] = 1
            else:
                action['camera'] = [0, 0]
                action['jump'] = 1
                action['forward'] = 1
                action['sprint'] = 1
        elif (env_name == 'MineRLTreechop-v0'):
            if (action_index == 0):
                action['camera'] = [0, -10]
                action['jump'] = 0
                action['forward'] = 1
                action['attack'] = 1
                action['sprint'] = 0
            elif (action_index == 1):
                action['camera'] = [0, 10]
                action['jump'] = 0
                action['forward'] = 1
                action['attack'] = 1
                action['sprint'] = 0
            elif (action_index == 2):
                action['camera'] = [-10, 0]
                action['jump'] = 0
                action['forward'] = 1
                action['attack'] = 1
                action['sprint'] = 0
            elif (action_index == 3):
                action['camera'] = [10, 0]
                action['jump'] = 0
                action['forward'] = 1
                action['attack'] = 1
                action['sprint'] = 0
            elif (action_index == 4):
                action['camera'] = [0, 0]
                action['jump'] = 0
                action['forward'] = 0
                action['attack'] = 1
                action['sprint'] = 0
            else:
                action['camera'] = [0, 0]
                action['jump'] = 1
                action['forward'] = 1
                action['attack'] = 1
                action['sprint'] = 0

                
            action['back'] = 0
            action['left'] = 0
            action['right'] = 0
            action['sprint'] = 0

        obs1, reward, done, info = env.step(action)
        
        if done == True:
            break
                
        obs = obs1

        net_reward += reward
    print("Total reward: ", net_reward)

Loading Model...
INFO:tensorflow:Restoring parameters from /home/kimbring2/MineRL/model/MineRLTreechop-v0/model-114300.cptk
Total reward:  7.0


As you can see, the network not only does much better than random actions, but achieves the goal of 200 points per episode, thus solving the task!